In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import requests
import re
import json
from pymongo import MongoClient
from odo import odo
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
lst = ['http://www.metmuseum.org/art/collection#!?material=Paintings&showOnly=withImage&offset=' + str(x) + '&pageSize=0&sortBy=Relevance&sortOrder=asc&perPage=20' for x in range(0, 12017, 20)]
for i in lst:
    print i

In [ ]:
fig_links = []

In [ ]:
mini_list = lst[522:]

In [ ]:
for i in mini_list:
    #page_count = 0
    driver = webdriver.PhantomJS()
    driver.get(i)
    time.sleep(2)
    #driver.quit()
    html = driver.page_source
    soup=BeautifulSoup(html)
    for figure in soup.find_all('figure',class_="card__standard-image"):
        #if i %20 == 0:
            #print'killing'
            #!killall phantomjs
        fig = figure.find('a')['href']
        fig_links.append(fig)
        with open('fig_links.txt', 'a') as myfile:
            myfile.write(fig)
        #print 'New PAGE!'
        #page_count += 1
        #print page_count
        driver.quit()

In [ ]:
my_links = []

for fig in fig_links:
    if '/art/collection/search/' in fig:
        url = 'http://www.metmuseum.org' + fig
        my_links.append(url)
        with open('my_links.txt', 'a') as myfile:
            myfile.write(url)
        print url

In [ ]:
len(my_links)/20 + 391

In [ ]:
with open('art_links') as f:
    data = f.readlines()
    data = [data.rstrip('\n') for data in open('art_links')]

data = filter(None, data)
data[:5]

In [ ]:
soup_objs = []

for line in data:
    response = requests.get(line)
    page = response.text
    link_soup = BeautifulSoup(page)
    soup_objs.append(link_soup)
    
response.status_code

In [ ]:
with open('my_soup.txt', 'a') as myfile:
    myfile.write(soup_objs)

In [ ]:
from collections import defaultdict

client = MongoClient()
db = client.paintings

artwork8 = db.artwork8


for soup in soup_objs: 
    art_dict = defaultdict(list)
    dets = soup.findAll('dt', class_='collection-details__tombstone--label')
    for s in dets:
        art_dict[s.text].append(s.nextSibling.nextSibling.text)
    art_dict['title'] = soup.find('h1', class_='collection-details__object-title').encode('utf-8')
    #art_dict['artist'] = soup.find('div',class_='collection-details__tombstone').encode('utf-8')
    art_dict['gallery'] = soup.find('div', class_='collection-details__location').encode('utf-8')
    art_dict['description'] = soup.find('div', class_='collection-details__label').encode('utf-8')
    images = soup.findAll('img')
    for i in images:
        if 'ng-src' in str(i):
            art_dict['image']=i.encode('utf-8')
    artwork8.insert_one(art_dict)

In [ ]:
artwork7.count()

In [ ]:
import pandas as pd

df = pd.DataFrame(list(artwork8.find()))

In [ ]:
df.to_csv('my_df.csv', sep='\t')

In [ ]:
def get_links(page):
    """given webpage get necassary links"""
    driver = webdriver.PhantomJS()
    driver.get(page)
    html = driver.page_source
    soup=BeautifulSoup(html)
    fig_links = []
    for figure in soup.find_all('figure',class_="card__standard-image"):
        fig = figure.find('a')['href']
        fig_links.append(fig)
        driver.quit()
        
    return fig_links

In [ ]:
def construct_url(link):
    """Given the link construct a valid URL"""
    my_links = []
    for fig in fig_links:
        if '/art/collection/search/' in fig:
            url = 'http://www.metmuseum.org' + fig
            my_links.append(url)
    return my_links

def get_data(url):
    """Given the url scrap required data"""
    response = requests.get(i)
    page = response.text
    link_soup = BeautifulSoup(page)
    soup_objs.append(link_soup)

In [ ]:
# MAIN
for page in lst:
    links = get_links(page)
    for i in links:
        url = construct_url(i)
        data = get_data(url)
        #write data to disk

In [ ]:
import numpy as np

df['description'] = df['description'].str.replace('<div class="collection-details__label">\n</div>', '')
df['description'] = df['description'].str.replace('<div class="collection-details__label">\r\n', '')
df['description'] = df['description'].str.replace('<br/><br/>', '')
df['description'] = df['description'].str.replace('\r\n    </div>', '')
df['gallery'] = df['gallery'].str.replace('<div class="collection-details__location">\n<svg class="icon">\n<use xlink:href="/dist/icons.svg#ico-marker" xmlns:xlink="http://www.w3.org/1999/xlink"></use>\n</svg>\r\n', '')
df['gallery'] = df['gallery'].str.replace('\r\n    </div>', '')
df['title'] = df['title'].str.replace('<h1 class="collection-details__object-title">', '')
df['title'] = df['title'].str.replace('</h1>', '')
#df['gallery'] = df['gallery'].str.replace('        On view at The Met Fifth Avenue in Gallery ', '')

## TEST

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import requests
import re
import json
from pymongo import MongoClient
from odo import odo
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
with open('art_links') as f:
    data = f.readlines()
    data = [data.rstrip('\n') for data in open('art_links')]

data = filter(None, data)
data = data[1600:1800]

In [ ]:
soup_objs = []

for line in data:
    response = requests.get(line)
    page = response.text
    link_soup = BeautifulSoup(page)
    soup_objs.append(link_soup)
    
response.status_code

In [ ]:
from collections import defaultdict

client = MongoClient()
db = client.paintings

artwork_test = db.artwork_test


for soup in soup_objs: 
    art_dict = defaultdict(list)
    dets = soup.findAll('dt', class_='collection-details__tombstone--label')
    for s in dets:
        art_dict[s.text].append(s.nextSibling.nextSibling.text)
    art_dict['title'] = soup.find('h1', class_='collection-details__object-title').encode('utf-8')
    #art_dict['artist'] = soup.find('div',class_='collection-details__tombstone').encode('utf-8')
    art_dict['gallery'] = soup.find('div', class_='collection-details__location').encode('utf-8')
    art_dict['description'] = soup.find('div', class_='collection-details__label').encode('utf-8')
    images = soup.findAll('img')
    for i in images:
        if 'ng-src' in str(i):
            art_dict['image']=i.encode('utf-8')
    artwork_test.insert_one(art_dict)

In [ ]:
artwork_test.count()

In [ ]:
import pandas as pd

df = pd.DataFrame(list(artwork_test.find()))

In [ ]:
df.head(500)